# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
raw_df = pd.read_csv('../output_data/WeatherPy_output/raw_weather.csv')
print(len(raw_df))
raw_df.head()

619


,id,main,description,icon,base,visibility,dt,timezone,name,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
0,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-10800.0,Belmonte,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,-36000.0,Hilo,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,803.0,Clouds,broken clouds,04n,stations,10000.0,1.608738e+09,3600.0,Honningsvåg,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,802.0,Clouds,scattered clouds,03d,stations,10000.0,1.608738e+09,-3600.0,Lagoa,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,801.0,Clouds,few clouds,02n,stations,9000.0,1.608738e+09,28800.0,George Town,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
import gmaps
gmaps.configure(api_key=g_key)

In [43]:
coords = raw_df[['coord.lat', 'coord.lon']].fillna(0)
humidity = raw_df['main.humidity']

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [296]:
ideal_temp_mask = (raw_df['main.temp_max'] < 80) & (raw_df['main.temp_max'] > 70)
ideal_wind_mask = raw_df['wind.speed'] < 10
ideal_cloud_mask = raw_df['clouds.all'] == 0 

ideal_spots_df = raw_df[ideal_temp_mask & ideal_wind_mask & ideal_cloud_mask]
ideal_spots_df.index = ideal_spots_df['name']
ideal_spots_df = ideal_spots_df.rename(columns={'name': 'City'})
ideal_spots_df.head()

,id,main,description,icon,base,visibility,dt,timezone,City,cod,...,filler_18,filler_19,filler_20,filler_21,filler_22,filler_23,filler_24,filler_25,filler_26,rain.1h
name,,,,,,,,,,,,,,,,,,,,,
Tessalit,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Tessalit,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bredasdorp,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,7200.0,Bredasdorp,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
São Pedro do Sul,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,-10800.0,São Pedro do Sul,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yulara,800.0,Clear,clear sky,01n,stations,10000.0,1.608738e+09,34200.0,Yulara,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Taoudenni,800.0,Clear,clear sky,01d,stations,10000.0,1.608738e+09,0.0,Taoudenni,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
ideal_coords = ideal_spots_df[['coord.lat','coord.lon']]
string_coords = []
responses = []

for vacation_spot in range(len(ideal_coords)):
    str_coord = f'{str(ideal_coords.iloc[vacation_spot, 0])}, {str(ideal_coords.iloc[vacation_spot,1])}'
    string_coords.append(str_coord)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

for vacation_spot in string_coords: 

    params = {
        "location": vacation_spot,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    responses.append(response)

In [252]:
clean_responses = [resp for resp in responses if resp.json()['status'] != 'ZERO_RESULTS']

In [291]:
from citipy import citipy

# Getting coords of the FIRST HOTEL OF EACH SEARCH WITHIN 5000m OF CITY 
hotel_info = {'Hotel Name': [],
              'Lat': [],
              'Lng': [],
              'City': []}
    
for result_list in clean_responses:
    hotels = result_list.json()
    first_hotel = hotels['results'][0]
    
    hotel_info['Hotel Name'].append(first_hotel['name'])
    hotel_info['Lat'].append(first_hotel['geometry']['location']['lat'])
    hotel_info['Lng'].append(first_hotel['geometry']['location']['lng'])

# Getting cities of hotels (hotel may not be within the city of search)
for coord_pair in zip(hotel_info['Lat'], hotel_info['Lng']):
    city = citipy.nearest_city(coord_pair[0], coord_pair[1]).city_name
    hotel_info['City'].append(city.title())

In [292]:
countries = ideal_spots_df[['City','sys.country']]
countries.head()

,City,sys.country
name,,
Tessalit,Tessalit,ML
Bredasdorp,Bredasdorp,ZA
São Pedro do Sul,São Pedro do Sul,BR
Yulara,Yulara,AU
Taoudenni,Taoudenni,ML


In [311]:
# Make hotel_df w/ hotel_name, lat, lng, country, city_name cols

hotel_df = pd.DataFrame(hotel_info)
hotel_df = pd.merge(hotel_df, countries, on='City',how='inner')
hotel_df = hotel_df.rename(columns={'sys.country': 'Country'})

hotel_df.head()

,Hotel Name,Lat,Lng,City,Country
0,Bredasdorp Square,-34.532655,20.039918,Bredasdorp,ZA
1,Sails in the Desert,-25.239022,130.983405,Yulara,AU
2,Hospitality Carnarvon,-24.892341,113.655034,Carnarvon,AU
3,Free Zone Hotel Nouadhibou,20.938950,-17.034546,Nouadhibou,MR
4,Hotel Castelo,-30.177881,-50.203501,Cidreira,BR


In [312]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [313]:
# Add marker layer ontop of heat map

hotel_names = hotel_df['Hotel Name'].values.tolist()

marker_layer = gmaps.marker_layer(locations, hotel_names, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))